In [66]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set()

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor, StackingRegressor

import pickle
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
import os
import joblib

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv('../data/original_data/listing.csv')

In [67]:
# PRICE DISTRIBUTION
# fe_df = df.copy()
# fe_df['price'] = pd.to_numeric(fe_df['price'].str.replace('[^.0-9]', ''))
# bins = np.arange(0,1000,50)
# plt.hist(np.clip(fe_df['price'], bins[0], bins[-1]), bins=bins)
# plt.xlabel('Price ($)')
# plt.show()

# CREATING FEATURE ENGINEERED DATASET USING ALL DATA

In [68]:
fe_df = df.copy()
fe_df['bathrooms'] = fe_df['bathrooms_text']
fe_df.drop(columns = 'bathrooms_text', inplace = True)

# change percentage columns to decimal
fe_df['host_response_rate'] = fe_df['host_response_rate'].str.rstrip('%').astype('float') / 100.0
fe_df['host_acceptance_rate'] = fe_df['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0

# change price column to float
fe_df['price'] = pd.to_numeric(fe_df['price'].str.replace('[^.0-9]', ''))

# map binary columns appropriately
fe_df['host_is_superhost'] = fe_df['host_is_superhost'].map({'f':0, 't':1})
fe_df['host_has_profile_pic'] = fe_df['host_has_profile_pic'].map({'f':0, 't':1})
fe_df['host_identity_verified'] = fe_df['host_identity_verified'].map({'f':0, 't':1})
fe_df['has_availability'] = fe_df['has_availability'].map({'f':0, 't':1})
fe_df['instant_bookable'] = fe_df['instant_bookable'].map({'f':0, 't':1})

# map date columns appropriately 
fe_df['host_since_year'] = pd.to_datetime(fe_df['host_since']).dt.year
fe_df['host_since'] = -(pd.to_datetime(fe_df['host_since']) -  pd.to_datetime("now")).dt.days
fe_df['first_review'] = -(pd.to_datetime(fe_df['first_review']) -  pd.to_datetime("now")).dt.days
fe_df['last_review'] = -(pd.to_datetime(fe_df['last_review']) -  pd.to_datetime("now")).dt.days

# clean bathroom column and convert to float
fe_df['bathrooms'] = fe_df['bathrooms'].str.extract(r'(\d+(?:\.\d+)?)').astype('float')

# impute room type depending on prop type
mask1 = (fe_df['property_type']==np.nan) & fe_df['room_type'].isnull()
mask2 = (fe_df['property_type'].isin(['Entire residential home', 'Entire townhouse', 'Entire cottage'])) & fe_df['room_type'].isnull()
mask3 = (fe_df['property_type']=='Room in boutique hotel') & fe_df['room_type'].isnull()

fe_df.loc[mask1, 'room_type'] = 'Private room'
fe_df.loc[mask2, 'room_type'] = 'Entire home/apt'
fe_df.loc[mask3, 'room_type'] = 'Hotel room'
fe_df['room_type'] = fe_df['room_type'].fillna('Private room') # anything else is priv room

# imputing missing cols with common sense substitution from other column
fe_df['minimum_minimum_nights'] = fe_df['minimum_minimum_nights'].fillna(fe_df['minimum_nights'])
fe_df['maximum_maximum_nights'] = fe_df['maximum_maximum_nights'].fillna(fe_df['maximum_nights'])
fe_df['availability_365'] = fe_df['availability_365'].fillna(fe_df['availability_90']*4) # can also fiddle with just 1 instead of 4 (did analysis)

# feature engineer for not reviewed before hosts
fe_df['host_has_not_accepted_before'] = (fe_df['host_acceptance_rate'].isnull()).astype(int)
fe_df['has_not_been_reviewed_before'] = (fe_df['first_review'].isnull()).astype(int)
fe_df['host_has_not_replied_before'] = (fe_df['host_response_rate'].isnull()).astype(int)

# impute nulls with 0 as context appropriate
# Edit: changed to 1
fe_df['bathrooms'] = fe_df['bathrooms'].fillna(1)

# fill na's for host response time
fe_df['host_response_time'] = fe_df['host_response_time'].fillna('no response time')

# host only has this listing
fe_df['host_single_listing'] = (fe_df['host_listings_count']==1).astype('int')

# review scores 
rev_cols = ['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
            'review_scores_communication', 'review_scores_location', 'review_scores_value']
for col in rev_cols:
    fe_df[col] = fe_df[col].fillna(value = fe_df['review_scores_rating'])
    
# bed and bedroom
bed_cols = ['beds', 'bedrooms']
for col in bed_cols:
    fe_df[col] = fe_df[col].fillna(value = 0) 

# convert license to binary var : exempt vs rest
fe_df['license_exempt_flag'] = fe_df['license'].map({'Exempt':1})
fe_df['license_exempt_flag'] = fe_df['license_exempt_flag'].fillna(0)
fe_df['PID_license_flag'] = [1 if x == True else 0 for x in fe_df["license"].str.lower().str.contains('pid')]
fe_df.drop(['license'], axis = 1, inplace = True)

# convert dates to time differences + add covid features
fe_df['host_joined_after_covid_flag'] = pd.to_datetime(fe_df['host_since']) > pd.to_datetime('2020-02-15')
fe_df['host_joined_after_covid_flag'] = fe_df['host_joined_after_covid_flag'].map({False:0, True:1})
fe_df['first_rev_after_covid_flag'] = pd.to_datetime(fe_df['first_review']) > pd.to_datetime('2020-02-15')
fe_df['first_rev_after_covid_flag'] = fe_df['first_rev_after_covid_flag'].map({False:0, True:1})

# fe_df['host_is_pro_managed'] = fe_df['host_name'].apply(check_host)
fe_df['host_neigh_provided'] = fe_df['host_neighbourhood'].isnull().astype('int')

# drop columns that will not be useful
fe_df.drop(['name','host_name', 'host_location', 'host_neighbourhood'], axis = 1, inplace = True)

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

# find missing vals in neighborhood cleansed col
missing_nb = fe_df[fe_df['neighbourhood_cleansed'].isna()][['neighbourhood_cleansed', 'neighbourhood', 'latitude', 'longitude']]

# create list of coordinates from the missing dataframe above
filled_nb = list(missing_nb['latitude'].astype('string') + "," + missing_nb['longitude'].astype('string'))
# drop neighbourhood col
fe_df.drop(['neighbourhood'], axis = 'columns', inplace = True)

In [69]:
walk_df = pd.read_excel('../data/walkscores.xlsx')
walk_df.columns = ['neighbourhood_cleansed', 'walkscore']
scores = fe_df.merge(walk_df, 'left')[['neighbourhood_cleansed', 'walkscore']]
fe_df['walkscore'] = scores['walkscore']

# feature engineering for verification column
fe_df['verf_cnt'] = [len(method) for method in fe_df["host_verifications"].str.lower().str.replace("\[|\]|\'", "").str.split(",").values]
fe_df['has_facebook'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('facebook')]
fe_df['has_jumio'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('jumio')]
fe_df['has_email'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('email')]
fe_df['has_review'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('review')]
fe_df['has_phone'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('phone')]
fe_df['has_google'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('google')]
fe_df['has_govt'] = [1 if x == True else 0 for x in fe_df["host_verifications"].str.lower().str.contains('government')]

# drop host verifications
fe_df.drop(['host_verifications'], axis = 'columns', inplace = True)

# feature engineering for property type
unique_props = ['barn', 'boat', 'cave', 'camper', 'farm stay', 'entire villa', 'entire residential home', 'aparthotel', 'hotel']
budget_props = ['bed and breakfast', 'hostel', 'earth', 'room in guesthouse', 'room in residential home']

fe_df['unique_prop_type'] = [1 if x == True else 0 for x in fe_df["property_type"].str.lower().str.contains('|'.join(unique_props))]
fe_df['budget_prop_type'] = [1 if x == True else 0 for x in fe_df["property_type"].str.lower().str.contains('|'.join(budget_props))]

# drop property type now
fe_df.drop('property_type', axis = 'columns', inplace = True)

# feature engineering 3 new cols - cnt of special amenities
hp_amens = ['parking on premise', 'fireplace', 'bathtub', 'private patio or balcony', 'pool', 'cable tv', 'bbq grill', 'hot tub', 
           'lake access', 'Pool table', 'Sonos Bluetooth sound system', 'Miele', 'walk-in closet', 'barbecue utensils', 
           'high chair', 'beach_essentials']
all_amens = ['Wifi', 'Long term stays allowed', 'Essentials', 'Smoke alarm','Kitchen', 'Washer', 'Hangers', 'Iron', 'Hair dryer', 'Shampoo',
       'Hot water', 'TV', 'Heating', 'Dedicated workspace','Dishes and silverware', 'Dryer', 'Refrigerator', 'Microwave',
       'Cooking basics', 'Air conditioning']
lp_amens = ['free street parking', 'luggage dropoff allowed', 'lock on bedroom', 'lockbox', 'shower gel', 'elevator', 'kettle', 'single level']

fe_df['highp_amen_cnt'] = 0
fe_df['pop_amen_cnt'] = 0
fe_df['lowp_amen_cnt'] = 0

for amen in hp_amens:
    fe_df.loc[fe_df['amenities'].str.lower().str.contains(amen.lower()), 'highp_amen_cnt'] += 1

for amen in all_amens:
    fe_df.loc[fe_df['amenities'].str.lower().str.contains(amen.lower()), 'pop_amen_cnt'] += 1
        
for amen in lp_amens:
    fe_df.loc[fe_df['amenities'].str.lower().str.contains(amen.lower()), 'lowp_amen_cnt'] += 1

# feature engineering for amenities
fe_df['amenities_cnt'] = [len(method) for method in fe_df["amenities"].str.replace("\[|\]|\'", "").str.split(",").values]

# add features for unique and cheap amenities

fe_df['amen_prem_park'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('parking on premise')]
fe_df['amen_ind_fireplace'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('fireplace')]
fe_df['amen_bathtub'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('bathtub')]
fe_df['amen_private_patio'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('private patio or balcony')]
fe_df['amen_pool'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('pool')]
fe_df['amen_cable'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('cable tv')]
fe_df['amen_bbq'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('bbq grill')]
fe_df['amen_hottub'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('hot tub')]
fe_df['amen_bbg_gear'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('barbecue utensils')]
fe_df['amen_child_toys'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('books and toys')]
fe_df['amen_high_chair'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('high chair')]
fe_df['amen_beach_ess'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('beach essentials')]

fe_df['amen_st_park'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('free street parking')]
fe_df['amen_lug_dropoff'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('luggage dropoff allowed')]
fe_df['amen_bedlock'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('lock on bedroom')]
fe_df['amen_lockbox'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('lockbox')]
fe_df['amen_elevator'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('elevator')]
fe_df['amen_singlev'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('single level')]
fe_df['amen_kettle'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('kettle')]
fe_df['amen_hostgreet'] = [1 if x == True else 0 for x in fe_df["amenities"].str.lower().str.contains('host greets you')]

# Calculate proportion of total amenities that are 'higher priced'
fe_df['highp_amen_prop'] = round(fe_df['highp_amen_cnt']/fe_df['amenities_cnt'],2)

# DUMMIES FOR TOP 60 MOST POPULAR SBURBS
pop_suburbs = list(fe_df['neighbourhood_cleansed'].value_counts()[:60].index)

for sub in pop_suburbs:
    fe_df['sub_' + sub.replace(" ", "_")] = [1 if x == True else 0 for x in fe_df["neighbourhood_cleansed"]==sub]

# creating a function for the next part
def clean_desc(series):
    """
    This function removes html text from a column (should be the description column)
    Inputs: 
        A series without null values
    Outputs:
        A bag of words from the series
    """
    html = ['<br /><br />', '<b>', '</b>', '<br />', 'Ä', 'ö', '√', 'Ñ', ';', ':', '<', '>', '/', '\\', '=', '-', '¥', 'Äö√Ñ√¥' ]
    for tag in html:
        series = series.str.replace(tag, '')
        
    return series

# feature engineering using above intuition
#fe_df['desc_beach'] = 
fe_df['description'] = clean_desc(fe_df['description'])

fe_df['description_length'] = fe_df['description'].map(str).apply(len)
fe_df['desc_beach'] = fe_df['description'].str.lower().str.contains('|'.join(['beach', 'water view', 'waterfront', 'ocean', 'coast'])).map({True:1, False:0, np.nan:0})
fe_df['desc_amen'] = fe_df['description'].str.lower().str.contains('|'.join(['spa', 'sauna', 'steam room', 'gym'])).map({True:1, False:0, np.nan:0})
fe_df['desc_view'] = fe_df['description'].str.lower().str.contains('|'.join(['view', 'sunset', 'sunrise', 'panorama', 'panoramic', 'infinity', 'levels', 'overlook'])).map({True:1, False:0, np.nan:0})
fe_df['desc_lux'] = fe_df['description'].str.lower().str.contains('|'.join(['estate','acre','lounge', 'deck', 'restaurant', 'balcony', 'renovated', 'modern', 'space', 'renovation', 'alfresco', 'designer'])).map({True:1, False:0, np.nan:0})
fe_df['desc_transport'] = fe_df['description'].str.lower().str.contains('|'.join(['minute', 'available', 'shared', 'walking','train', 'station', 'bus', 'cbd'])).map({True:1, False:0, np.nan:0})


# note: the below were gotten from the correlation matrix as 0 corr 
    # https://vishesh-gupta.medium.com/correlation-in-xgboost-8afa649bd066
    # https://stats.stackexchange.com/questions/266267/should-one-be-concerned-about-multi-collinearity-when-using-non-linear-models

cols_to_drop = ['description', 'neighborhood_overview', 'neighbourhood_cleansed', 'host_about', 'amenities', 'host_picture_url',
                'id', 'listing_url', 'scrape_id', 'last_scraped', 'picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'calendar_last_scraped',
                'calendar_updated', 'neighbourhood_group_cleansed', 
                ]

# note: the below were gotten from the correlation matrix as 0 corr 
    # https://vishesh-gupta.medium.com/correlation-in-xgboost-8afa649bd066
    # https://stats.stackexchange.com/questions/266267/should-one-be-concerned-about-multi-collinearity-when-using-non-linear-models
dist_df = pd.read_excel('final_distances.xlsx', index_col = 0)
hway_dist_df = pd.read_excel('hway_distances.xlsx', index_col = 0)

fe_df2 = fe_df.drop(cols_to_drop, axis = 1).copy()
fe_df_dums = pd.get_dummies(fe_df2)
fe_df_dums = fe_df_dums.join(dist_df['dist']).copy()
hway_dist_df.columns = ['hway_dist']
fe_df_dums = fe_df_dums.join(hway_dist_df['hway_dist']).copy()

Sentiment analysis took 10 min  
Final distances took 9 mins  
Highway distances took 2 mins 

In [ ]:
# # THE CODE BELOW TAKES A WHILE TO RUN HENCE I HAVE USED EXCEL INSTEAD
# # Get sentiment score for description and neighboorhood overview
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# desc_sentiment = [] # description sentiment
# for i in fe_df['description']:
#     desc_sentiment.append(SentimentIntensityAnalyzer().polarity_scores(str(i))['compound'])
# fe_df['desc_sentiment'] = pd.Series(desc_sentiment)
# neigh_sentiment = [] # neighbourhood sentiment
# for i in fe_df['neighborhood_overview']:
#     neigh_sentiment.append(SentimentIntensityAnalyzer().polarity_scores(str(i))['compound'])
# fe_df['neigh_sentiment'] = pd.Series(neigh_sentiment)
# host_neigh_sent = [] # host neighbourhood sentiment
# for i in fe_df['host_about']:
#     host_neigh_sent.append(SentimentIntensityAnalyzer().polarity_scores(str(i))['compound'])
# fe_df['host_about_sentiment'] = pd.Series(host_neigh_sent)

# fe_df[['desc_sentiment', 'neigh_sentiment', 'host_about_sentiment']].to_excel('sentiment_scores.xlsx')

# exp_lat = [-33.977696,
# -33.983353,
# -33.989201,
# -33.991842,
# -33.978549,
# -33.964214,
# -33.939094,
# -33.935071,
# -33.930274,
# -33.920968,
# -33.915001,
# -33.915849,
# -33.913614,
# -33.911022,
# -33.906451,
# -33.903587,
# -33.891459,
# -33.892676,
# -33.879146,
# -33.873718,
# -33.865029,
# -33.859503,
# -33.850456,
# -33.844058,
# -33.835353,
# -33.839878,
# -33.843046,
# -33.847018,
# -33.852903,
# -33.850651,
# -33.868973,
# -33.869690,
# -33.865112,
# -33.873992,
# -33.866709,
# -33.870952,
# -33.870530,
# -33.867814,
# -33.863572,
# -33.863264,
# -33.857650,
# -33.882320,
# -33.857132,
# -33.929798,
# -33.889944,
# -33.867841,
# -33.851706,
# -33.848177,
# -33.845931,
# -33.842259,
# -33.847712,
# -33.845917,
# -33.842103,
# -33.842174,
# -33.822557,
# -33.815283,
# -33.819420,
# -33.806476,
# -33.809528,
# -33.802982,
# -33.798890,
# -33.800174,
# -33.807448,
# -33.818812,
# -33.796970,
# -33.801214,
# -33.781750,
# -33.776296,
# -33.771480,
# -33.763881,
# -33.757994,
# -33.753890,
# -33.737696,
# -33.712838,
# -33.699854,
# -33.676179,
# -33.656264,
# -33.648387,
# -33.643315,
# -33.635445,
# -33.629578,
# -33.610213,
# -33.597467,
# -33.601720,
# -33.592676,
# -33.616697,
# -33.636034,
# -33.648470,
# -33.658635,
# -33.663135,
# -33.663420,
# -33.661062,
# -33.655525,
# -33.650202,
# -33.652795,
# -33.644935,
# -33.89318,
# -33.63151,
# -33.77904,
# -33.82199,
# -33.51341,
# -33.75837,
# -33.78945,
# -33.867448,
# -33.866719,
# -33.874093,
# -33.879372,
# -33.85503,
# -33.840582,
# -33.596905,
# -33.816517,
# -34.008981,
# -34.013238,
# -34.04521,
# -34.013287,
# -34.017857,
# -34.007698,
# -33.991254,
# -33.998121,
# -34.078135,
# -34.081702,
# -34.08401,
# -33.766286,
# -33.517781]

# exp_lon = [151.228079,
# 151.230515,
# 151.235563,
# 151.238403,
# 151.251755,
# 151.251985,
# 151.262270,
# 151.261380,
# 151.259680,
# 151.257626,
# 151.261164,
# 151.267136,
# 151.266393,
# 151.270119,
# 151.269946,
# 151.268228,
# 151.276042,
# 151.284334,
# 151.282448,
# 151.283521,
# 151.283440,
# 151.284655,
# 151.286993,
# 151.284930,
# 151.280265,
# 151.278882,
# 151.281823,
# 151.280447,
# 151.273639,
# 151.266851,
# 151.268921,
# 151.253410,
# 151.252892,
# 151.244632,
# 151.239971,
# 151.235620,
# 151.230513,
# 151.227327,
# 151.226874,
# 151.217714,
# 151.214644,
# 151.263990,
# 151.268733,
# 151.259667,
# 151.278405,
# 151.238425,
# 151.218060,
# 151.219562,
# 151.224025,
# 151.216129,
# 151.203984,
# 151.229883,
# 151.233531,
# 151.250998,
# 151.251008,
# 151.248691,
# 151.223979,
# 151.229773,
# 151.237887,
# 151.236042,
# 151.282680,
# 151.273797,
# 151.284354,
# 151.293565,
# 151.288467,
# 151.297651,
# 151.290060,
# 151.293345,
# 151.294203,
# 151.301670,
# 151.299910,
# 151.297206,
# 151.306128,
# 151.301321,
# 151.308997,
# 151.315598,
# 151.323064,
# 151.326921,
# 151.330597,
# 151.332489,
# 151.339247,
# 151.331517,
# 151.324709,
# 151.316941,
# 151.320675,
# 151.317227,
# 151.306587,
# 151.302993,
# 151.307918,
# 151.310278,
# 151.305815,
# 151.300923,
# 151.293713,
# 151.286889,
# 151.277286,
# 151.283766,
# 151.26875,
# 151.31457,
# 151.12056,
# 151.19739,
# 150.90451,
# 151.24892,
# 151.28486,
# 151.250007,
# 151.253684,
# 151.245861,
# 151.250331,
# 151.274354,
# 151.248566,
# 151.324065,
# 151.249134,
# 151.191907,
# 151.202607,
# 151.159248,
# 151.113493,
# 151.115124,
# 151.086539,
# 151.128968,
# 151.135473,
# 151.129342,
# 151.14592,
# 151.152535,
# 150.653403,
# 151.176154]

# exp_coords = pd.DataFrame({
#     'latitude':exp_lat,
#     'longitude':exp_lon
# })

# dist_list = []
# import geopy.distance
# hot_locs = exp_coords.copy()
# all_coords = fe_df[['latitude', 'longitude']].copy()
# for index, row in all_coords.iterrows():
#     airbnb_loc = (row['latitude'], row['longitude'])
#     # this is our constant to compare against so we can grab min in below loop
#     starting_diff = geopy.distance.geodesic(airbnb_loc, (hot_locs.loc[0]['latitude'], hot_locs.loc[0]['longitude'])).km
#     # will use this to change and appen to min dist list
#     diff = starting_diff
#     for index, row in hot_locs.iterrows():
#         hotloc_coord = (row['latitude'], row['longitude'])
#         if geopy.distance.geodesic(airbnb_loc, hotloc_coord).km < diff:
#             diff = round(geopy.distance.geodesic(airbnb_loc, hotloc_coord).km,3)
#     dist_list.append(diff)
    
# dist_df = pd.DataFrame(dist_list)
# dist_df.columns = ['dist']
# dist_df.to_excel('final_distances.xlsx')
# #dist_df = pd.read_excel('final_distances.xlsx', index_col = 0)

# mtrway_lat = [
#     -33.829912,
# -33.842568,
# -33.857561,
# -33.865162,
# -33.877167,
# -33.81623,
# -33.797706,
# -33.758232,
# -33.754883,
# -33.738626,
# -33.751075,
# -33.950768,
# -33.938074,
# -33.925247,
# -33.941688
# ]

# mtrway_lon = [
#     151.018103,
# 151.043708,
# 151.072131,
# 151.094252,
# 151.12962,
# 150.963331,
# 150.854017,
# 151.049254,
# 150.953926,
# 150.917531,
# 150.845879,
# 150.876465,
# 150.913459,
# 150.921265,
# 150.942502
# ]

# highway_coords = pd.DataFrame({
#     'latitude':mtrway_lat,
#     'longitude':mtrway_lon
# })

# dist_list = []
# import geopy.distance
# all_coords = fe_df[['latitude', 'longitude']].copy()

# for index, row in all_coords.iterrows():
#     airbnb_loc = (row['latitude'], row['longitude'])
#     # this is our constant to compare against so we can grab min in below loop
#     starting_diff = geopy.distance.geodesic(airbnb_loc, (highway_coords.loc[0]['latitude'], highway_coords.loc[0]['longitude'])).km
#     # will use this to change and appen to min dist list
#     diff = starting_diff
    
#     for index, row in highway_coords.iterrows():
#         hotloc_coord = (row['latitude'], row['longitude'])
#         if geopy.distance.geodesic(airbnb_loc, hotloc_coord).km < diff:
#             diff = round(geopy.distance.geodesic(airbnb_loc, hotloc_coord).km,3)
    
#     dist_list.append(diff)
    
# hway_dist_df = pd.DataFrame(dist_list)
# hway_dist_df.columns = ['hway_dist']
# hway_dist_df.to_excel('hway_distances.xlsx')



# cols_to_drop = ['description', 'neighborhood_overview', 'neighbourhood_cleansed', 'host_about', 'amenities']

In [83]:
x_fe = fe_df_dums.drop(columns = 'price').copy()
y_fe = fe_df_dums['price'].copy()

print(x_fe.shape, y_fe.shape)

(20880, 151) (20880,)


In [98]:
default_df = pd.read_csv('../data/original_data/listing.csv')
# clean bathroom column and convert to float
default_df['bathrooms'] = default_df['bathrooms_text']
default_df.drop(columns = 'bathrooms_text', inplace = True)

# drop text or useless cols
cols_to_drop = ['id','description','name', 'neighborhood_overview', 'host_name',
                'host_location','host_about','host_neighbourhood','host_verifications',
                'host_picture_url','id', 'listing_url', 'scrape_id', 'last_scraped', 'picture_url', 
                'host_id', 'host_url', 'host_thumbnail_url', 'calendar_last_scraped', 'calendar_updated', 'neighbourhood_group_cleansed', 
                'neighbourhood','property_type','amenities','license']
default_df.drop(columns = cols_to_drop, inplace=True)

# fix price
default_df['price'] = pd.to_numeric(default_df['price'].str.replace('[^.0-9]', ''))

# format percentages 
default_df['host_response_rate'] = default_df['host_response_rate'].str.rstrip('%').astype('float') / 100.0
default_df['host_acceptance_rate'] = default_df['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0

# map binary columns appropriately
default_df['host_is_superhost'] = default_df['host_is_superhost'].map({'f':0, 't':1})
default_df['host_has_profile_pic'] = default_df['host_has_profile_pic'].map({'f':0, 't':1})
default_df['host_identity_verified'] = default_df['host_identity_verified'].map({'f':0, 't':1})
default_df['has_availability'] = default_df['has_availability'].map({'f':0, 't':1})
default_df['instant_bookable'] = default_df['instant_bookable'].map({'f':0, 't':1})

# map date columns appropriately 
default_df['host_since_year'] = pd.to_datetime(default_df['host_since']).dt.year
default_df['host_since'] = -(pd.to_datetime(default_df['host_since']) -  pd.to_datetime("now")).dt.days
default_df['first_review'] = -(pd.to_datetime(default_df['first_review']) -  pd.to_datetime("now")).dt.days
default_df['last_review'] = -(pd.to_datetime(default_df['last_review']) -  pd.to_datetime("now")).dt.days

# imputing missing cols with common sense substitution from other column
default_df['minimum_minimum_nights'] = default_df['minimum_minimum_nights'].fillna(default_df['minimum_nights'])
default_df['maximum_maximum_nights'] = default_df['maximum_maximum_nights'].fillna(default_df['maximum_nights'])
default_df['availability_365'] = default_df['availability_365'].fillna(default_df['availability_90']*4) # can also fiddle with just 1 instead of 4 (did analysis)

default_df = pd.get_dummies(default_df)
default_df.shape

(20880, 125)

In [ ]:
# fix gremlins
y_pred[y_pred<20] = 20